In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf

from sklearn.model_selection import train_test_split

import data_utils

In [ ]:
dataset = data_utils.open_pickle('data/ludb_processed/ludb.pickle')

X = dataset['X']
y = dataset['y']

#### Splitting X and y to train, validation, and test set

In [ ]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=.2, shuffle=False, random_state=2023)
X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=.5, shuffle=False, random_state=2023)

len(X_train), len(X_val), len(X_test)

In [ ]:
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)

In [ ]:
def generate_model(input_shape, output, lr=1e-5, n_layer=1):
    model = tf.keras.models.Sequential()
    opt = tf.keras.optimizers.Adam(learning_rate=lr)

    # Conv layers
    model.add(tf.keras.layers.Conv1D(8, 3, input_shape=input_shape, padding='same', activation='relu'))
    model.add(tf.keras.layers.Conv1D(16, 3, padding='same', activation='relu'))
    model.add(tf.keras.layers.Conv1D(32, 3, padding='same', activation='relu'))
    model.add(tf.keras.layers.Conv1D(64, 3, padding='same', activation='relu'))

    # Bidirectional LSTM
    model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(512, input_shape=input_shape, return_sequences=True)))

    for i in range(n_layer-1):
        model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(512, return_sequences=True)))

    # Fully connected layers
    model.add(tf.keras.layers.Dense(output, activation='softmax'))
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

    return model

In [ ]:
model = generate_model((1387, 1), 7)
model.summary()

In [ ]:
model.fit(X_train, y_train, epochs=100, batch_size=8, validation_data=(X_val, y_val))